In [33]:
%pip install -U pip
%pip install scikit-learn pandas numpy joblib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# 0. 개요: 스키마(JSON/TXT) 입력을 받아 6개 카테고리 중 1개를 예측하는 단일 스크립트
# - 학습(train): 폴더 내 .json/.txt 재귀 수집 → TF-IDF(char 3~5) + LinearSVC → 저장
# - 예측(predict): 입력(.json/.txt, 단일/리스트) 또는 STDIN → 라벨 1개(싱글라벨) 출력
# - 전처리: 최소(필드 결합 + 공백 정규화만)

# 1. 입력
import os, re, json, glob, argparse, joblib, sys
from typing import List, Dict, Any, Iterable
import numpy as np
import pandas as pd
from collections import Counter

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

LABELS: List[str] = ["기타","대회","모집","자금","진로","행사"]


# 2. 로더(.json/.txt)
def read_json_text(path: str) -> Any:
    """파일을 문자열로 읽어 JSON 파싱(.json/.txt 공통)."""
    with open(path, "r", encoding="utf-8-sig") as f:
        raw = f.read()
    return json.loads(raw)

def load_records_from_path(path: str) -> List[Dict[str, Any]]:
    """단일 객체/리스트를 레코드 리스트로 변환."""
    data = read_json_text(path)
    return data if isinstance(data, list) else [data]

def iter_schema_files(root: str) -> Iterable[str]:
    """폴더 내 스키마 파일(.json/.txt) 재귀 탐색."""
    patterns = ["**/*.json", "**/*.txt"]
    for pat in patterns:
        for p in glob.glob(os.path.join(root, pat), recursive=True):
            yield p


# 3. 특징 구성(스키마 필드 → 하나의 텍스트)
def safe_join(v: Any) -> str:
    """리스트/딕셔너리/스칼라를 문자열로 안전 결합."""
    if v is None: return ""
    if isinstance(v, str): return v
    if isinstance(v, (int, float, bool)): return str(v)
    if isinstance(v, list): return " ".join(safe_join(x) for x in v)
    if isinstance(v, dict): return " ".join(safe_join(x) for x in v.values())
    return str(v)

def extract_text(rec: Dict[str, Any]) -> str:
    """스키마 주요 필드를 하나의 텍스트로 단순 결합(공백 정규화만)."""
    parts = []
    for key in ["제목","설명","세부카테고리","주최기관"]:
        if key in rec: parts.append(safe_join(rec[key]))
    if "대상" in rec and isinstance(rec["대상"], dict):
        for k in ["연령","지역","특이조건"]:
            if k in rec["대상"]: parts.append(safe_join(rec["대상"][k]))
    if "기간" in rec and isinstance(rec["기간"], dict):
        for k in ["start","end"]:
            if k in rec["기간"] and rec["기간"][k] is not None:
                parts.append(str(rec["기간"][k]))
    return re.sub(r"\s+"," ", " ".join(parts)).strip()

def collect_dataset(data_dir: str) -> pd.DataFrame:
    """폴더 내 .json/.txt를 읽어 (text,label) 데이터프레임 생성."""
    rows = []
    for path in iter_schema_files(data_dir):
        try:
            for rec in load_records_from_path(path):
                text = extract_text(rec)
                label = rec.get("카테고리", None)
                if text and label in LABELS:
                    rows.append({"path": path, "text": text, "label": label})
        except Exception:
            # 깨진 파일은 건너뜀(필요 시 로깅)
            pass
    return pd.DataFrame(rows)


# 4. 모델 파이프라인
def build_pipeline(class_weight: dict = None) -> Pipeline:
    return Pipeline([
        ("tfidf", TfidfVectorizer(
            analyzer="char_wb",
            ngram_range=(3,5),
            min_df=1,
            max_df=0.95
        )),
        ("clf", LinearSVC(class_weight=class_weight))
    ])



# 5. 학습/검증/저장
def train_model(
    data_dir: str,
    model_out: str,
    labels_out: str,
    test_size: float = 0.2,
    kfold: int = 0,
    report_out: str = None
):
    """폴더 스캔→데이터 수집→학습/평가→저장."""
    df = collect_dataset(data_dir)
    if df.empty:
        raise RuntimeError("학습할 JSON/TXT가 없습니다. 스키마와 '카테고리' 값을 확인하세요.")
    X, y = df["text"].values, df["label"].values

    # 등장 라벨 기준 가중치 계산(불균형 보정)
    present = np.array(sorted(df["label"].unique().tolist()))
    cw = compute_class_weight(class_weight="balanced", classes=present, y=y)
    class_weight = {c: w for c, w in zip(present, cw)}
    class_weight["자금"] = class_weight.get("자금", 1.0) * 3.0  # 데이터 불균형으로 인한 조정

    # 홀드아웃
    pipe = build_pipeline(class_weight)
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=test_size, random_state=42, stratify=y
    )
    pipe.fit(X_tr, y_tr)
    y_pr = pipe.predict(X_te)

    acc = accuracy_score(y_te, y_pr)
    f1m = f1_score(y_te, y_pr, average="macro", zero_division=0)
    print(f"[HOLD-OUT] ACC={acc:.4f}  Macro-F1={f1m:.4f}")
    print(classification_report(y_te, y_pr, digits=4, zero_division=0))
    print("Confusion Matrix\n", confusion_matrix(y_te, y_pr, labels=LABELS))

    # K-Fold(선택) — 소수 클래스 샘플 수보다 큰 K는 자동 감소
    if kfold and kfold > 1:
        counts = Counter(y)
        min_count = min(counts.values())
        if min_count < kfold:
            print(f"[INFO] 일부 클래스 샘플이 {min_count}개여서 K={kfold}→{min_count}로 조정합니다.")
            kfold = min_count
        if kfold > 1:
            skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=42)
            accs, f1s = [], []
            for i, (tr, va) in enumerate(skf.split(X, y), 1):
                p = build_pipeline(class_weight); p.fit(X[tr], y[tr]); pr = p.predict(X[va])
                accs.append(accuracy_score(y[va], pr))
                f1s.append(f1_score(y[va], pr, average="macro", zero_division=0))
                print(f"[K{i}] ACC={accs[-1]:.4f}  Macro-F1={f1s[-1]:.4f}")
            print(f"[K-AVG] ACC={np.mean(accs):.4f}  Macro-F1={np.mean(f1s):.4f}")

    # 저장
    joblib.dump(pipe, model_out)
    with open(labels_out, "w", encoding="utf-8") as f:
        json.dump({"labels": LABELS}, f, ensure_ascii=False, indent=2)

    if report_out:
        with open(report_out, "w", encoding="utf-8") as f:
            f.write(f"[HOLD-OUT] ACC={acc:.6f}  Macro-F1={f1m:.6f}\n")
            f.write(classification_report(y_te, y_pr, digits=4, zero_division=0))


# 6. 예측(파일/STDIN/파이썬 객체)
def predict_records(model_path: str, in_json: Any) -> List[str]:
    """모델 로드 후 레코드 리스트에 대해 싱글라벨 예측."""
    pipe: Pipeline = joblib.load(model_path)
    recs = in_json if isinstance(in_json, list) else [in_json]
    texts = [extract_text(r) for r in recs]
    preds = pipe.predict(texts)
    return preds.tolist()


# 7. CLI
def main():
    """argparse 기반 CLI 엔트리포인트."""
    ap = argparse.ArgumentParser(description="스키마(JSON/TXT) → 6카테고리 싱글라벨 분류")
    sub = ap.add_subparsers(dest="cmd")  # 주피터 대비: required=False

    # 학습
    ap_tr = sub.add_parser("train", help="폴더 스캔 후 학습/평가/저장")
    ap_tr.add_argument("--data_dir", default=".", help="학습용 폴더(재귀 검색)")
    ap_tr.add_argument("--model_out", default="json_schema_classifier.pkl")
    ap_tr.add_argument("--labels_out", default="labels.json")
    ap_tr.add_argument("--report_out", default=None)
    ap_tr.add_argument("--test_size", type=float, default=0.2)
    ap_tr.add_argument("--kfold", type=int, default=0, help="교차검증 K(0=미사용)")

    # 예측
    ap_pr = sub.add_parser("predict", help="입력 파일(.json/.txt) 또는 STDIN 예측")
    ap_pr.add_argument("--model", required=True, help="학습된 모델(.pkl)")
    ap_pr.add_argument("--input", default=None, help="입력 파일(생략 시 STDIN 사용)")

    args = ap.parse_args()

    # 서브커맨드 없을 때: 노트북이면 조용히 리턴, 터미널이면 도움말 출력
    if args.cmd is None:
        if _running_in_notebook():
            print("[INFO] Notebook에서 실행됨: CLI는 건너뛰고 함수만 정의했습니다.")
            return
        else:
            ap.print_help()
            sys.exit(2)

    if args.cmd == "train":
        train_model(
            data_dir=args.data_dir,
            model_out=args.model_out,
            labels_out=args.labels_out,
            test_size=args.test_size,
            kfold=args.kfold,
            report_out=args.report_out
        )
        print("모델 저장:", args.model_out)
        print("라벨 저장:", args.labels_out)

    elif args.cmd == "predict":
        if args.input:
            data = read_json_text(args.input)
        else:
            data = json.loads(sys.stdin.read())
        preds = predict_records(args.model, data)
        if isinstance(data, list):
            for i, p in enumerate(preds, 1):
                print(f"[{i}] {p}")
        else:
            print(preds[0])


# 8. 실행 가드
def _running_in_notebook() -> bool:
    try:
        from IPython import get_ipython  # noqa
        return get_ipython() is not None
    except Exception:
        return False

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] {train,predict} ...
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\User\AppData\Roaming\jupyter\runtime\kernel-v302755f17b5e78c933f37bd66249c3b135a2a3fd3.json


SystemExit: 2

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [64]:
# 예시: 주피터에서 함수 직접 호출 (여러 파일 일괄 예측)
DATA_DIR   = "../json/all_json_data"   # 학습 데이터 폴더
MODEL_OUT  = "model.pkl"
LABELS_OUT = "labels.json"
REPORT_OUT = "report.txt"

# 학습
train_model(DATA_DIR, MODEL_OUT, LABELS_OUT, test_size=0.2, kfold=10, report_out=REPORT_OUT)

[HOLD-OUT] ACC=0.8033  Macro-F1=0.7291
              precision    recall  f1-score   support

          기타     1.0000    0.4615    0.6316        13
          대회     0.8000    1.0000    0.8889        32
          모집     0.7333    0.6111    0.6667        18
          진로     0.8077    0.9545    0.8750        44
          행사     0.7778    0.4667    0.5833        15

    accuracy                         0.8033       122
   macro avg     0.8238    0.6988    0.7291       122
weighted avg     0.8115    0.8033    0.7861       122

Confusion Matrix
 [[ 6  4  1  0  1  1]
 [ 0 32  0  0  0  0]
 [ 0  1 11  0  5  1]
 [ 0  0  0  0  0  0]
 [ 0  0  2  0 42  0]
 [ 0  3  1  0  4  7]]
[K1] ACC=0.8361  Macro-F1=0.7961
[K2] ACC=0.7869  Macro-F1=0.7216
[K3] ACC=0.7705  Macro-F1=0.7023
[K4] ACC=0.8852  Macro-F1=0.8305
[K5] ACC=0.8033  Macro-F1=0.7452
[K6] ACC=0.8361  Macro-F1=0.7908
[K7] ACC=0.8361  Macro-F1=0.7232
[K8] ACC=0.7705  Macro-F1=0.6149
[K9] ACC=0.7833  Macro-F1=0.6810
[K10] ACC=0.7667  Macro-F1=0.6

In [65]:
# 예측 대상: 디렉터리 내 모든 .json/.txt
import os, glob

PRED_ROOT = "../json/pred_sample_data"  # 예측 대상 파일들이 있는 폴더
pred_paths = sorted(
    glob.glob(os.path.join(PRED_ROOT, "**/*.json"), recursive=True) +
    glob.glob(os.path.join(PRED_ROOT, "**/*.txt"),  recursive=True)
)

for path in pred_paths:
    try:
        data  = read_json_text(path)              # 단일 JSON 또는 JSON 리스트
        preds = predict_records(MODEL_OUT, data)  # 항상 리스트 반환
        if isinstance(data, list):
            for i, p in enumerate(preds, 1):
                print(f"{path} [{i}] -> {p}")
        else:
            print(f"{path} -> {preds[0]}")
    except Exception as e:
        print(f"{path} -> ERR: {e}")


../json/pred_sample_data\46991_2024_07_10_111420_진로.json -> 진로
../json/pred_sample_data\47305_2024_08_12_144312_대회.json -> 대회
../json/pred_sample_data\47337_2024_08_14_112922_행사.json -> 행사
../json/pred_sample_data\47377_2024_08_19_113108_행사.json -> 행사
../json/pred_sample_data\47424_2024_08_22_171004_대회.json -> 대회
../json/pred_sample_data\47425_2024_08_22_171131_기타.json -> 기타
../json/pred_sample_data\47432_2024_08_23_095239_진로.json -> 진로
../json/pred_sample_data\48139_2024_10_29_103055_자금.txt -> 모집
../json/pred_sample_data\48359_2024_11_12_114556_모집.txt -> 모집
../json/pred_sample_data\48418_2024_11_18_154647_모집.txt -> 모집
../json/pred_sample_data\48834_2024_12_27_131548_자금.txt -> 모집
../json/pred_sample_data\49840_2025_04_14_141550_기타.json -> 기타
